# Hyperbolic Partial Differential Equations

## Advection Equation

Here we solve the advection equation 
$$
\frac{\partial u}{\partial t} + a \frac{\partial u}{\partial x}=0
$$ 

using as an initial condition a gaussian profile $u(x,t=0)=\exp{[-(x-x_0)^2]}$ with $x_0=5$.

We use a domain $x\in [0,10]$ with periodic boundary conditions.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#define the speed a
a=1.0

# Define the domain
L = 10.0     # Domain length
nx = 101    # Number of grid points
dx = L/(nx-1)   # Grid spacing
x = np.linspace(0, L, nx)

# Define the time step and the final time
cf = 0.5
dt = cf*dx/a   # Time step
t_final = 20.0

print('nx=',nx)
print('dx=',dx)
print('dt=',dt)
print('Number of iterations=',t_final/dt)


# Define the initial condition
x0=5
u_initial = np.exp(-(x-x0)**2)

# Initialize the solution array
u_current = u_initial.copy()

#we create arrays where we will store the time and the l2norm
l2norm=[]
l2norm.append(np.sqrt(np.sum(u_current**2)/len(u_current)))

time=[]
time.append(0.0)

#we plot the initial data
plt.ylim(-0.1,1.1)
plt.plot(x, u_current)
plt.show()

In [ ]:
import os
##create directory where to save images
print(os.getcwd())

os.makedirs('./images')

os.listdir('./')

### FTCS

The Forward in Time - Centered in Space (FTCS) is the first scheme we are going to use. It is first order in time and second order in space:

$$
u^{n+1}_j = u^n_j - \frac{a\Delta t}{2\Delta x} \left( u^n_{j+1} - u^n_{j-1}  \right)
$$

In [ ]:
# Initilize time and iteration counter
t = 0.0
i = 0

#save the initial conditions
plt.plot(x, u_current)
plt.title('Time='+str(round(t,2)))
plt.ylim(-0.1,1.1)
plt.savefig('./images/fig_'+str(i).zfill(5)+'.png', dpi=200)
plt.close()

#solve the advection equation
while t < t_final:
    # Compute the new solution using the FTCS method
    # Note: np.roll(u_current, -1) is equivalent to u(j+1) and
    #       np.roll(u_current,  1) is equivalent to u(j-1)
    # using np.roll is equivalent to use periodic boundary conditions
    u_next = u_current - a*dt/(2*dx)*(np.roll(u_current, -1) - np.roll(u_current, 1))
    
    # Update the solution
    u_current = u_next.copy()
    
    
    #advance the time 
    t += dt
    i += 1
    
    #compute the l2 norm and add the time to the time vector
    l2norm.append(np.sqrt(np.sum(u_current**2)/len(u_current)))
    time.append(t)
    
    #plot the current result and save in an image every 10 iterations
    if (i%10==0):
        plt.plot(x, u_current)
        plt.title('Time='+str(round(t,2)))
        plt.ylim(-0.1,1.1)
        plt.savefig('./images/fig_'+str(i).zfill(5)+'.png', dpi=200)
        plt.close()


In [ ]:
# Plot the final solution
plt.plot(x, u_initial, label='Initial')
plt.plot(x, u_current, label='Final')
plt.title('Time='+str(round(t,2)))
plt.ylim(-0.1,1.1)
plt.legend()
plt.show()

In [ ]:
# set the directory where your images are stored
directory = "./images/"

# get the list of image files in the directory
files = os.listdir(directory)

print(files, '\n')

# sort the files in alphanumeric order
files=sorted(files)

print(files)




In [ ]:
import imageio
with imageio.get_writer('./movie.mp4', mode='I', quality=10) as writer:
    for file in files:
        image = imageio.imread('./images/'+file)
        writer.append_data(image)
        
files=[]


In [ ]:
# don't worry about the code in this cell, it is just to let you 
# display the movies you generated above in Jupyter notebook
from IPython.display import HTML

HTML("""
<div align="middle">
<video width="80%" controls>
      <source src="./movie.mp4" type="video/mp4">
</video></div>""")

In [ ]:
plt.plot(time,l2norm)
plt.show()

As you can see this method is unstable since it develops oscillations that grow in time and destroy the solution.

You can try instead to use the following methods that are stable as long as the Courant-Friedrichs-Lewy (CFL) condition is satisfied:
$$
\frac{|a|\Delta t}{\Delta x} \leq 1
$$

This condition essentially requires that your numerical domain of dependence is larger than the physical domain of dependence.

### Lax-Friedrichs

This method is also first order like the FTCS, but it adds a dissipative term to remove the instabilities present in the FTCS scheme:

$$
u^{n+1}_j = \frac{1}{2}\left( u^n_{j-1}+u^n_{j+1} \right) - \frac{a\Delta t}{2\Delta x} \left( u^n_{j+1} - u^n_{j-1}  \right)
$$

### Upwind

This method is also first order, but it is less dissipative than Lax-Friedrichs. It has two versions depending on the sign of $a$:

1. if $a>0$
    $$
    u^{n+1}_j = u^n_j - \frac{a\Delta t}{\Delta x} \left( u^n_{j} - u^n_{j-1}  \right)
    $$
    
2. if $a<0$
    $$
    u^{n+1}_j = u^n_j - \frac{a\Delta t}{\Delta x} \left( u^n_{j+1} - u^n_{j}  \right)
    $$

### Leapfrog

This method is second order, but it requires three timelevels:

$$
u^{n+1}_j = u^{n-1}_j - \frac{a\Delta t}{\Delta x} \left( u^n_{j+1} - u^n_{j-1}  \right)
$$

### Lax-Wendroff

This method is also second order, but it requires only two timelevels making its implementation easier than Leapfrog:

$$
u^{n+1}_j = u^n_j - \frac{a \Delta t}{2\Delta x} \left( u^n_{j+1} - u^n_{j-1}  \right) + \frac{a^2 \Delta t^2}{2\Delta x^2} \left( u^n_{j+1} - 2 u^n_j + u^n_{j-1}  \right)
$$

### Exercise

Use two of the above methods to solve the advection equation and check what happens when increasing the resolution (use at least three different resolutions). Plot the solution $u(x,t)$ as a function of $x$ at different times, inclduding $t=0$ and $t=20$. Plot also the l2-norm and comment on what you observe.

## Step Function

Solve the advection equation using a step function for the initial data:

$$
u(x,t=0) = 1 \quad \mathrm{for}\quad x \in [4,6]\\
u(x,t=0) = 0 \quad \mathrm{otherwise} 
$$

Compare the first-order **upwind** scheme with the second-order **Lax-Wendroff** scheme. Which one is more accurate in this case?